<a href="https://colab.research.google.com/github/megboes/groves-research/blob/main/printing_Grove_polynomials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import matplotlib.pyplot as plt
import math
import sympy as sp

max_num_boxes=4

pi_0=[[]]
pi_1=[[[0,0,0]]]
pi_2=[[[0,0,0],[0,0,1]],[[0,0,0],[0,1,0]],[[0,0,0],[1,0,0]]]
master_pi=[pi_0,pi_1,pi_2]


In [84]:
def check_if_box_should_be_added(box, partition):
  we_should_add=True
  a,b,c=box
  if box in partition:
    we_should_add=False
  if (a>=1)and([a-1,b,c] in partition)==False:
    we_should_add=False
  if (b>=1)and([a,b-1,c] in partition)==False:
    we_should_add=False
  if (c>=1)and([a,b,c-1] in partition)==False:
    we_should_add=False
  return(we_should_add)

def given_single_partitions_produces_list_of_bigger_partitions(given_partition):
  some_partitions_of_more_boxes = []
  for element in given_partition:
    a,b,c=element
    candidate=[a+1,b,c]
    result=check_if_box_should_be_added(candidate,given_partition)
    if result==True:
      pi_plus = given_partition+[candidate]
      some_partitions_of_more_boxes.append(pi_plus)

  for element in given_partition:
    a,b,c=element
    candidate=[a,b+1,c]
    result=check_if_box_should_be_added(candidate,given_partition)
    if result==True:
      pi_plus = given_partition+[candidate]
      some_partitions_of_more_boxes.append(pi_plus)

  for element in given_partition:
    a,b,c=element
    candidate=[a,b,c+1]
    result=check_if_box_should_be_added(candidate,given_partition)
    if result==True:
      pi_plus = given_partition+[candidate]
      some_partitions_of_more_boxes.append(pi_plus)
  return(some_partitions_of_more_boxes)

def deletes_duplicates(collection_of_partitions):
  trial=collection_of_partitions
  for element in trial:
   number=trial.count(element)
   while number >=2:
     trial.remove(element)
     number=trial.count(element)
  collection_of_partitions=trial
  return(collection_of_partitions)

In [85]:
def given_partition_determines_height(partition):
  #find size of space needed for grove
  height=0
  for box in partition:
    a,b,c=box
    height = max(height, a+b+c)
  height+=2
  return(height)

In [86]:
def generates_bigger_master_pi(master_pi):

################################### next block generates paritions up to max_num_boxes #################################3
#yeah, this should be it's own function
  for i in range(2,max_num_boxes):
    for part in master_pi[i]:
        if len(master_pi)==i+1:
          master_pi.append(given_single_partitions_produces_list_of_bigger_partitions(part))
        elif len(master_pi)==i+2:
          master_pi[i+1]+=given_single_partitions_produces_list_of_bigger_partitions(part)
        else: print('something went wrong')
    for part in master_pi[i+1]:
      part.sort()
      master_pi[i+1].sort()
    master_pi[i+1] = deletes_duplicates(master_pi[i+1])
  return(master_pi)
################ paritions up to max_num_boxes, i.e. MASTER_PI complete ###########################3


In [87]:
master_pi=generates_bigger_master_pi(master_pi)

In [88]:
def given_partition_and_height_determines_initial_conditions(partition, height):
  #set everything in the space to 0
  initial_conditions = []

  for i in range(height):
    initial_conditions.append([])
    for j in range(height):
      initial_conditions[i].append([])
      for k in range(height):
        initial_conditions[i][j].append(0)
  # so now all coordinats have value of 0

  # here we make some coordinates equal to 1
  for box in partition:
    a,b,c=box
    if [a,b,c+1] not in partition:
      initial_conditions[a][b][c+1]=sp.symbols('x_{'+str(a)+str(b)+str(c+1)+'}')
      initial_conditions[a][b+1][c+1]=sp.symbols('x_{'+str(a)+str(b+1)+str(c+1)+'}')
      initial_conditions[a+1][b][c+1]=sp.symbols('x_{'+str(a+1)+str(b)+str(c+1)+'}')
      initial_conditions[a+1][b+1][c+1]=sp.symbols('x_{'+str(a+1)+str(b+1)+str(c+1)+'}')
    if [a,b+1,c] not in partition:
      initial_conditions[a][b+1][c]=sp.symbols('x_{'+str(a)+str(b+1)+str(c)+'}')
      initial_conditions[a][b+1][c+1]=sp.symbols('x_{'+str(a)+str(b+1)+str(c+1)+'}')
      initial_conditions[a+1][b+1][c]=sp.symbols('x_{'+str(a+1)+str(b+1)+str(c)+'}')
      initial_conditions[a+1][b+1][c+1]=sp.symbols('x_{'+str(a+1)+str(b+1)+str(c+1)+'}')
    if [a+1,b,c] not in partition:
      initial_conditions[a+1][b][c]=sp.symbols('x_{'+str(a+1)+str(b)+str(c)+'}')
      initial_conditions[a+1][b][c+1]=sp.symbols('x_{'+str(a+1)+str(b)+str(c+1)+'}')
      initial_conditions[a+1][b+1][c]=sp.symbols('x_{'+str(a+1)+str(b+1)+str(c)+'}')
      initial_conditions[a+1][b+1][c+1]=sp.symbols('x_{'+str(a+1)+str(b+1)+str(c+1)+'}')
    #mission complete!
  return(initial_conditions)




In [89]:
def given_initial_conditions_and_height_determines_grove_poly(conditions, height):
  cg=conditions
  #add in a, b, c variables
  a=[]
  for j in range(height):
    a.append([])
    for k in range(height):
      a[j].append(sp.symbols('a_{'+str(j)+str(k)+'}'))
  b=[]
  for i in range(height):
    b.append([])
    for k in range(height):
      b[i].append(sp.symbols('b_{'+str(i)+str(k)+'}'))
  c=[]
  for i in range(height):
    c.append([])
    for j in range(height):
      c[i].append(sp.symbols('c_{'+str(i)+str(j)+'}'))

  while cg[0][0][0]==0:
    for i in range(height-1):
      for j in range(height-1):
        for k in range(height-1):
          if (cg[i][j][k]==0) and (cg[i+1][j][k]!=0) and (cg[i][j+1][k]!=0) and (cg[i][j][k+1]!=0):
            cg[i][j][k]=(sp.symbols('z_{'+str(i)+str(j)+str(k)+'}'))*(b[i][k]*c[i][j]*cg[i+1][j][k]*cg[i][j+1][k+1]+c[i][j]*a[j][k]*cg[i][j+1][k]*cg[i+1][j][k+1]+a[j][k]*b[i][k]*cg[i][j][k+1]*cg[i+1][j+1][k])/cg[i+1][j+1][k+1]
  poly=sp.expand(cg[0][0][0])
  #print(poly)
  return(poly)

In [90]:
#see if we can get this one to take in a partition, and returna list of monomials

def takes_partition_returns_list_of_monomials(pi):
  poly =given_partition_prints_grove(pi)
  monomials=poly.as_ordered_terms()
  # for monomial in monomials:
  #   print(monomial)
  # print(monomials)
  return(monomials)

In [91]:
def given_partition_prints_grove(pi):
  tall = given_partition_determines_height(pi)
  g = given_partition_and_height_determines_initial_conditions(pi, tall)
  f=given_initial_conditions_and_height_determines_grove_poly(g, tall)
  print(f)
  return(f)

In [92]:
for i in range(1,len(master_pi)):
  for partition in master_pi[i]:
    print("For the partition "+str(partition)+" we have the polynomial: ")
    pile_of_monos = takes_partition_returns_list_of_monomials(partition)
    print("which gives us the the monomials/groves:")
    for j in range(len(pile_of_monos)):
      print(pile_of_monos[j])




For the partition [[0, 0, 0]] we have the polynomial: 
a_{00}*b_{00}*x_{001}*x_{110}*z_{000}/x_{111} + a_{00}*c_{00}*x_{010}*x_{101}*z_{000}/x_{111} + b_{00}*c_{00}*x_{011}*x_{100}*z_{000}/x_{111}
which gives us the the monomials/groves:
a_{00}*b_{00}*x_{001}*x_{110}*z_{000}/x_{111}
a_{00}*c_{00}*x_{010}*x_{101}*z_{000}/x_{111}
b_{00}*c_{00}*x_{011}*x_{100}*z_{000}/x_{111}
For the partition [[0, 0, 0], [0, 0, 1]] we have the polynomial: 
a_{00}*a_{01}*b_{00}*b_{01}*x_{002}*x_{110}*z_{000}*z_{001}/x_{112} + a_{00}*a_{01}*b_{00}*c_{00}*x_{011}*x_{102}*x_{110}*z_{000}*z_{001}/(x_{111}*x_{112}) + a_{00}*b_{00}*b_{01}*c_{00}*x_{012}*x_{101}*x_{110}*z_{000}*z_{001}/(x_{111}*x_{112}) + a_{00}*c_{00}*x_{010}*x_{101}*z_{000}/x_{111} + b_{00}*c_{00}*x_{011}*x_{100}*z_{000}/x_{111}
which gives us the the monomials/groves:
a_{00}*a_{01}*b_{00}*b_{01}*x_{002}*x_{110}*z_{000}*z_{001}/x_{112}
a_{00}*a_{01}*b_{00}*c_{00}*x_{011}*x_{102}*x_{110}*z_{000}*z_{001}/(x_{111}*x_{112})
a_{00}*b_{00}*b_{01}*c_